# Behavior analysis of cFos experiment


In [ ]:
import socket
import os
import pandas as pd

In [ ]:
if 'startDirMaster' not in locals():
    startDirMaster=os.getcwd()

propsFn=startDirMaster+'\\props.csv'

props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()
#print(props)
props

In [ ]:
codeDir = '..\\jlsocialbehavior'
    

if 'NB_path' not in locals():
    NB_path = os.getcwd()
else:
    os.chdir(NB_path)
    

base=props['BaseDir']
#artDir=props['ArtDir']
RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'+'behavior\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'
FigureDir = os.path.join(base,props['FigureDir'])+'\\'

expFile=props['allExpBehFn']
anFile=props['allAnBehFn']


props

print('baseFolder = ' + base)



os.chdir(codeDir)
props

In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import fnmatch
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
from datetime import datetime
from scipy import stats
import statsmodels.stats.api as sms

import functions.matrixUtilities_joh as mu
#import functions.notebookHelper as nh
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
import functions.peakdet as pkd

import matplotlib as mpl
mpl.rcParams['svg.fonttype'] = 'none'

pfp.paper()
inToCm=2.54

In [ ]:
expFile

In [ ]:
info=pd.read_excel(expFile)
ix=(info.stimulusProtocol=='cFos')#&(info.date.isin(['2019-08-05','2019-08-06','2019-09-25','2019-10-04']))
info=info[ix].reset_index(drop=True)
info

In [ ]:
infoAn=pd.read_excel(anFile)
infoAn.tail()

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
birthDayAll=[]
anIDsAll=[]
camHeightAll=[]

camHeight=[105,180] #for arena up,dn

for index,row in info.iterrows():

    startDir=RawDataDir+row.path+'\\'
    if not os.path.isdir(startDir):
        print(startDir)
        raise FileNotFoundError 
    
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
    camHeightAll.append(camHeight[('_dn_' in head)*1])
    
    anNrs=row.anNr #Note that anNrs are 1 based!
    if ':' in anNrs:
        a,b=anNrs.split(sep=':')
        anNrs=np.arange(int(a),int(b)+1)
    else:
        anNrs=np.array(anNrs.split()).astype(int)
        
    anIDs=anNrs #-1 no more 0-based since using pandas merge to find animal numbers
    anIDsAll.extend(anIDs)

    bd=infoAn[infoAn.anNr.isin(anIDs)].bd.values
    #bd=infoAn.bd.values[anIDs-1] #a bit dirty to use anIDs directly here. Should merge
    birthDayAll.append(' '.join(list(bd)))

info['camHeight']=camHeightAll
info['txtPath']=posPath
info['pairList']=PLPath
info['aviPath']='default'
info['birthDayAll']=birthDayAll
info['epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
info['episodes'] = 9   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = 70 # arena diameter (default: 100 mm)
info['minShift'] = 120 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 0 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 1 # flag to compute swim bout frequency (takes time, default: 1)
#info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir
info['expTime']=expTime
info['readLim'] = 54*5*60*30+11

In [ ]:
startDir

In [ ]:
csvFile=os.path.join(ProcessingDir,'processingSettings.csv')
info.to_csv(csvFile,encoding='utf-8')
info.tail()

In [ ]:
info.shape

In [ ]:
rereadData=1
if rereadData:
    def readExperiment(keepData=True):
        tmp=es.experiment_set(csvFile=csvFile,MissingOnly=True)
        if keepData:
            return tmp
        else:
            return 1

    expSet=readExperiment(keepData=True)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    tmp.animalIndex=tmp.animalIndex+((i)*35)
    tmp.animalIndex=np.array(anIDsAll)[tmp.animalIndex]
    df=pd.concat([df,tmp])
    i+=1
df['episode']=[x.strip().replace('_','') for x in df['episode']]
df=pd.merge(df,infoAn[['anNr','line','group']],left_on='animalIndex',right_on='anNr',how='left')
df=pd.merge(df,info[['date']],left_on='animalSet',right_on=info.index,how='left')

print('df shape',df.shape)
df.tail(10)

In [ ]:
df.info()

# Overview attraction over time

Plot shoaling index over time for each group.

In [ ]:
dfPlot=(df.groupby(['inDishTime','group']).si.agg(['mean','std'])
    .unstack()
    .stack(dropna=True)
    .reset_index())

dfPlot.head()


In [ ]:
fig, axes = plt.subplots(figsize=(6, 3))
g=sns.scatterplot(data=dfPlot,x='inDishTime',hue='group',y='mean')
plt.xlim([0,60])
plt.ylim([-.05,.4])
plt.xlabel('Time (Minutes)')
plt.ylabel('Attraction')
plt.title('Mean attraction, all animals')
#plt.legend(title='Stimulus dot motion')
#plt.legend(labels=['Continuous motion','Bout motion','Continuous + loom','Bout + loom'],
#          title='Stimulus dot motion')

#new_title = 'Stimulus dot motion'
#g.legend_.set_title(new_title)
# replace labels
plt.legend(ncol=1,handletextpad=0,bbox_to_anchor=(1, 1),loc='upper left')
plt.axhline(0)

sns.despine()

figPath=base+'ScreenTimeCourse.png'
#plt.savefig(figPath,bbox_inches='tight')


# Mean response over all stimuli per animal

In [ ]:
df.animalSet.unique()

In [ ]:
sns.set_palette('viridis',3)
co=sns.color_palette("viridis", 3)
idx=(df['inDishTime']<60) & (df['inDishTime']>15)
dfDR=df[idx]
dfEpiAn=dfDR.groupby(['episode','animalIndex','line','group','date'],sort=True).mean().reset_index()

In [ ]:
sns.stripplot(data=dfEpiAn,x='group',y='si',zorder=-1,hue='line')
sns.pointplot(data=dfEpiAn,x='group',y='si',hue='line',zorder=100,scale=0.2,palette=['gray'],join=False)
sns.pointplot(data=dfEpiAn,x='group',y='si',join=False,zorder=100)
ax=plt.gca()
ax.legend_.remove()

In [ ]:
fig, axes = plt.subplots(figsize=(3,3))
sns.pointplot(data=dfEpiAn,
              x='group',
              y='si',
              hue='group',
              zorder=100,
              scale=1,
              join=False,
             ci='sd',
             dodge=.5)
sns.despine()

axes.set_ylabel('attraction index')
axes.axhline(0,ls=':',color='k')
axes.set_title('Attraction tuning per stim group');
plt.legend(title='age',ncol=2,handletextpad=0,bbox_to_anchor=(1, 1.05))
plt.xticks(rotation=90);

In [ ]:
fig, axes = plt.subplots(figsize=(3,3))
sns.pointplot(data=dfEpiAn,
              x='group',
              y='avgSpeed_smooth',
              hue='group',
              zorder=100,
              scale=1,
              join=False,
             ci='sd',
             dodge=.5)
sns.despine()

axes.set_ylabel('speed')
axes.axhline(0,ls=':',color='k')
axes.set_title('Attraction tuning per stim group');
plt.legend(title='age',ncol=2,handletextpad=0,bbox_to_anchor=(1, 1.05))
plt.xticks(rotation=90);

In [ ]:
fig, axes = plt.subplots(figsize=(3,3))
sns.pointplot(data=dfEpiAn,
              x='group',
              y='boutDur',
              hue='group',
              zorder=100,
              scale=1,
              join=False,
             ci='sd',
             dodge=.5)
sns.despine()

axes.set_ylabel('boutDur')
axes.axhline(0,ls=':',color='k')
axes.set_title('Attraction tuning per stim group');
plt.legend(title='age',ncol=2,handletextpad=0,bbox_to_anchor=(1, 1.05))
plt.xticks(rotation=90);

In [ ]:
co=sns.color_palette("tab10", 4)
co[1:]=co[:-1]
co[0]=[.6,.6,.6]
co

In [ ]:
order=['invi','cont','bout','fish']
fig, axes = plt.subplots(figsize=(4/inToCm, 9/inToCm))
ix=(dfEpiAn.episode=='02k20f')#&(~dfEpiAn.lineSet.str.contains('_2_'))

selDat=dfEpiAn
#co=sns.color_palette("Dark2", 4)
#co[0]=[.6,.6,.6]
#co[1]=[0,0,.9]
#co[2]=[.95,0,0]
#co[3]=[0,.6,0]

sns.swarmplot(data=selDat,
              x='group',
              y='si',
              hue='group',
              zorder=-1,
              dodge=.5,
              size=5,
              alpha=.7,
              order=order,
              hue_order=order,
              palette=co
             )

sns.pointplot(data=selDat,
              x='group',
              y='si',
              hue='group',
              zorder=1000,
              size=8,
              dodge=.5,
              join=False,
              color='k',
              ci='sd',
              order=order,
              hue_order=order,
             )

sns.despine()

axes.set_ylabel('Attraction')
axes.set_xticklabels('')
axes.set_xticks([])
axes.set_xlabel('')

axes.axhline(0,ls=':',color='k')
handles, labels = axes.get_legend_handles_labels()

labels=np.array(['Bout-like','Continuous','Conspecific','No stim'])[[3,1,0,2]]

l = plt.legend(handles[0:4], labels[0:4], title='',ncol=2,handletextpad=0,
               loc='upper center',bbox_to_anchor=(0.5, 1.2),
                             labelspacing=0,
               borderpad=0,
               handlelength=1.5,
               columnspacing=0.1,
              frameon=0,
              fontsize=10)

plt.xlim([-1,4])
plt.ylim([-.1,.5])
axes.text(.5,.42,'ns')
axes.text(1.6,.45,'***')
axes.text(2.8,.48,'***')

for handle in l.legendHandles:
    handle.set_sizes([12.0])

#plt.rcParams['svg.fonttype'] = 'path'
figPath=FigureDir+'hcrBehavior.svg'
plt.savefig(figPath,bbox_inches='tight')


In [ ]:
import scipy.stats as sta
def groupttest(x, cat='group',test=2):
    dat=x.copy()
    dat.set_index(['animalID', cat])

    catLevels = np.array(['invi', 'cont', 'bout', 'fish'])
    res=[]
    for i,c in enumerate(catLevels[1:]):
        d2 = dat[dat[cat] == c].si#iloc[:,2:]
        d1 = dat[dat[cat] == 'invi'].si#iloc[:,2:]
        s,p=sta.ttest_ind(d1,d2)
        #print(c,d1.mean(),d2.mean(),d1.mean()-d2.mean(),p)
        res.append(p)
    return np.array(res)

In [ ]:
groupttest(x=selDat[['group','animalID','si']],cat='group')

In [ ]:
dat=selDat[['group','animalID','si']]
dat.set_index(['animalID', 'group'])
cat='group'
catLevels = np.array(['invi', 'cont', 'bout', 'fish'])
res=[]
dat[dat[cat] == 'invi'].si

In [ ]:
x=selDat.groupby(['group']).si.mean().values
x=x[[3,1,0,2]]
selDat.groupby(['group']).si.mean()

In [ ]:
y=np.array([0.5,1.1,1,1.5])


In [ ]:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

print(x)
print(y)
plt.plot(x,y,'.--')

print(slope, intercept, r_value, p_value, std_err)